# Project: (K-) Nearest Neighbors


## Programming project: probability of death

In this project, you have to predict the probability of death of a patient that is entering an ICU (Intensive Care Unit).

The dataset comes from MIMIC project (https://mimic.physionet.org/). MIMIC-III (Medical Information Mart for Intensive Care III) is a large, freely-available database comprising deidentified health-related data associated with over forty thousand patients who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012.

Each row of *mimic_train.csv* correponds to one ICU stay (*hadm_id*+*icustay_id*) of one patient (*subject_id*). Column HOSPITAL_EXPIRE_FLAG is the indicator of death (=1) as a result of the current hospital stay; this is the outcome to predict in our modelling exercise.
The remaining columns correspond to vitals of each patient (when entering the ICU), plus some general characteristics (age, gender, etc.), and their explanation can be found at *mimic_patient_metadata.csv*. 

Please don't use any feature that you infer you don't know the first day of a patient in an ICU.

Note that the main cause/disease of patient condition is embedded as a code at *ICD9_diagnosis* column. The meaning of this code can be found at *MIMIC_metadata_diagnose.csv*. **But** this is only the main one; a patient can have co-occurrent diseases (comorbidities). These secondary codes can be found at *extra_data/MIMIC_diagnoses.csv*.

As performance metric, you can use *AUC* for the binary classification case, but feel free to report as well any other metric if you can justify that is particularly suitable for this case.

Main tasks are:
+ Using *mimic_train.csv* file build a predictive model for *HOSPITAL_EXPIRE_FLAG* .
+ For this analysis there is an extra test dataset, *mimic_test_death.csv*. Apply your final model to this extra dataset and generate predictions following the same format as *mimic_kaggle_death_sample_submission.csv*. Once ready, you can submit to our Kaggle competition and iterate to improve the accuracy.

As a *bonus*, try different algorithms for neighbor search and for distance, and justify final selection. Try also different weights to cope with class imbalance and also to balance neighbor proximity. Try to assess somehow confidence interval of predictions.

You can follow those **steps** in your first implementation:
1. *Explore* and understand the dataset. 
2. Manage missing data.
2. Manage categorial features. E.g. create *dummy variables* for relevant categorical features, or build an ad hoc distance function.
3. Build a prediction model. Try to improve it using methods to tackle class imbalance.
5. Assess expected accuracy  of previous models using *cross-validation*. 
6. Test the performance on the test file and report accuracy, following same preparation steps (missing data, dummies, etc). Remember that you should be able to yield a prediction for all the rows of the test dataset.

Feel free to reduce the training dataset if you experience computational constraints.

## Main criteria for IN_CLASS grading
The weighting of these components will vary between the in-class and extended projects:
+ Code runs - 20%
+ Data preparation - 35%
+ Nearest neighbor method(s) have been used - 15%
+ Probability of death for each test patient is computed - 10%
+ Accuracy of predictions for test patients is calculated (kaggle) - 10%
+ Hyperparameter optimization - 5%
+ Class imbalance management - 5%
+ Neat and understandable code, with some titles and comments - 0%
+ Improved methods from what we discussed in class (properly explained/justified) - 0%

# Setup

## Import packages and functions

In [ ]:
#packages

import pandas as pd
import numpy as np 
import os
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
from datetime import datetime 
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix
from category_encoders import TargetEncoder, BinaryEncoder, WOEEncoder
from sklearn.svm import SVC
import pickle

In [ ]:
#functions  

def plot_confusion_matrix(cm, class_labels):
    """Pretty prints a confusion matrix as a figure

    Args:
        cm:  A confusion matrix for example
        [[245, 5 ], 
         [ 34, 245]]
         
        class_labels: The list of class labels to be plotted on x-y axis

    Rerturns:
        Just plots the confusion matrix.
    """
    
    df_cm = pd.DataFrame(cm, index = [i for i in class_labels],
                  columns = [i for i in class_labels])
    sns.set(font_scale=1)
    sns.heatmap(df_cm, annot=True, fmt='g', cmap='Blues')
    plt.xlabel("Predicted label")
    plt.ylabel("Real label")
    plt.show()

def getting_dates(DOBs, ATs):
  admit_dates = []
  birthdays = []
  for i in range(0, len(DOBs)):
      birthdays.append(datetime.strptime(DOBs[i], '%Y-%m-%d %H:%M:%S'))
  for j in range(0, len(ATs)):
      admit_dates.append(datetime.strptime(ATs[j], '%Y-%m-%d %H:%M:%S'))
  return birthdays, admit_dates

def getting_age(birthdays, admit_dates):
  ages = []
  for i in range(0, len(birthdays)):
    ages.append(((admit_dates[i] - birthdays[i]).days)/365.25)
  return ages

def age_fix(data, feature):
    for i in range(0, len(data[feature])):
        if data.loc[i, feature] > 120:
            data.loc[i, feature] = 95
    return data

def drop_feature(features, data):
    for feature in features:
        data = data.drop(feature, axis = 1)
    return data 

def one_hot_encode(features, data):
    for feature in features:
        data = pd.get_dummies(data, prefix=[feature], columns=[feature], drop_first = True)
    return data

def replace(data, feature_to_replace, feature_replacements, new_feature):
    data[feature_to_replace] = data[feature_to_replace].replace(feature_replacements, new_feature)
    return data

def reweight_binary(pi,q1=0.5,r1=0.5):
    r0 = 1-r1
    q0 = 1-q1
    tot = pi*(q1/r1)+(1-pi)*(q0/r0)
    w = pi*(q1/r1)
    w /= tot
    return w

In [ ]:
#setup and load data

data_path = '/Users/benseimon/Documents/Barca GSE/Studies/Term 2/CML2/Project 1/Data'
kaggle_path = '/Users/benseimon/Documents/Barca GSE/Studies/Term 2/CML2/Project 1/Kaggle submissions'
model_path = '/Users/benseimon/Documents/Barca GSE/Studies/Term 2/CML2/Project 1/Models'
os.chdir(data_path)
comorbidities = pd.read_csv('MIMIC_diagnoses.csv')
diagnosis_definitions = pd.read_csv('MIMIC_metadata_diagnose.csv')
feature_definitions = pd.read_excel('mimic_patient_metadata.xlsx')
train_data = pd.read_csv('mimic_train.csv')
test_data = pd.read_csv('mimic_test_death.csv')

# Step 1: Exploratory Data Analysis

In [ ]:
#drop as per instructions - note that offending columns are not in test set so no need to drop 
features_to_drop = ['DOD', 'DISCHTIME', 'DEATHTIME', 'LOS', 'Diff']
train_data = train_data.drop(features_to_drop, axis=1)
test_data = test_data.drop('Diff', axis=1)

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

## 1a: Numerical features

### 1ai: Generate age variable

Using 'DOB' and 'ADMITTIME' we create an age variable. Note that using 'Diff is not necessary since you would simply apply diff to both columns. 

According to this source (https://github.com/MIT-LCP/mimic-code/issues/637) the data anonymisation process assigns anyone older than 89 to an age close to 300. As a result, I replace any realistic ages to age of 95 (arbitrary). 

Note that purpose of creating the variable is driven by a simple hypothesis that old age increases the likelihood of dying. 

In [ ]:
#create age variable for training set
DoB_train, admit_date_train = getting_dates(train_data['DOB'], train_data['ADMITTIME'])
ages = getting_age(DoB_train, admit_date_train)
train_data['AGE'] = ages
print("Distribution before amending those with an unrealistic age")
print(train_data['AGE'].describe())
train_data = age_fix(train_data, 'AGE')
train_data = drop_feature(['DOB', 'ADMITTIME'], train_data)
print("Distribution after amending those with an unrealistic age")
print(train_data['AGE'].describe())

In [ ]:
#create age variable for test set
DOB_test, admit_date_test = getting_dates(test_data['DOB'], test_data['ADMITTIME'])
ages = getting_age(DOB_test, admit_date_test)
test_data['AGE'] = ages
print("Distribution before amending those with an unrealistic age")
print(test_data['AGE'].describe())
test_data = age_fix(test_data, 'AGE')
test_data = drop_feature(['DOB', 'ADMITTIME'], test_data)
print("Distribution after amending those with an unrealistic age")
print(test_data['AGE'].describe())

### 1aii: Investigate relationship between outcome and numerical variables

In [ ]:
#assign features
identifiers = ['subject_id', 'hadm_id', 'icustay_id']
numerical_features = ['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min',
       'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean',
       'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min',
       'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean',
       'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max',
       'Glucose_Mean', 'AGE']
categorical_features = ['GENDER', 'ADMISSION_TYPE', 'INSURANCE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'FIRST_CAREUNIT']
diagnosis_features = ['DIAGNOSIS','ICD9_diagnosis']
target = ['HOSPITAL_EXPIRE_FLAG']

In [ ]:
#check
len(identifiers+numerical_features+categorical_features+diagnosis_features+target) == train_data.shape[1]

In [ ]:
corr_numerical = train_data[numerical_features+target].corr()
sns.set(rc = {'figure.figsize':(20,10)})
sns.heatmap(corr_numerical, cmap = 'RdBu_r', linecolor = 'white', vmin=-1, vmax=1, annot = True)

In [ ]:
corr_numerical['HOSPITAL_EXPIRE_FLAG'].sort_values(ascending = False)

Clear positive correlation for RespRate_Mean, RespRate_Max, HeartRate_Max, HeartRate_Mean, Glucose_Mean and our new variable AGE

Clear negative correlation for DiasBP_Mean, TempC_Mean, MeanBP_Mean, SysBP_Mean, TempC_Min, DiasBP_Min,           SpO2_Mean, MeanBP_Min, SysBP_Min, SpO2_Min

## Categorical features

In [ ]:
train_data[categorical_features].nunique()

Ethnicity and religion have a higher number of unique categories. Let's address this to reduce dimensionality. 

### Ethnicities

In [ ]:
train_data['ETHNICITY'].value_counts()

In [ ]:
#categorise ethnicities 
NATIVE = ['AMERICAN INDIAN/ALASKA NATIVE',
 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE',  'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER']
ASIAN = ['ASIAN',
 'ASIAN - ASIAN INDIAN',
 'ASIAN - CAMBODIAN',
 'ASIAN - CHINESE',
 'ASIAN - FILIPINO',
 'ASIAN - JAPANESE',
 'ASIAN - KOREAN',
 'ASIAN - OTHER',
 'ASIAN - THAI',
 'ASIAN - VIETNAMESE']
BLACK = ['BLACK/AFRICAN',
 'BLACK/AFRICAN AMERICAN',
 'BLACK/CAPE VERDEAN',
 'BLACK/HAITIAN']
OTHER = ['CARIBBEAN ISLAND', 'OTHER', 'MIDDLE EASTERN','MULTI RACE ETHNICITY']

HISPANIC_LATINO = ['HISPANIC OR LATINO',
 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)',
 'HISPANIC/LATINO - COLOMBIAN',
 'HISPANIC/LATINO - CUBAN',
 'HISPANIC/LATINO - DOMINICAN',
 'HISPANIC/LATINO - GUATEMALAN',
 'HISPANIC/LATINO - HONDURAN',
 'HISPANIC/LATINO - MEXICAN',
 'HISPANIC/LATINO - PUERTO RICAN',
 'HISPANIC/LATINO - SALVADORAN', 
                   'SOUTH AMERICAN']
UNKNOWN = ['UNABLE TO OBTAIN',
 'UNKNOWN/NOT SPECIFIED']
WHITE = ['WHITE',
 'WHITE - BRAZILIAN',
 'WHITE - EASTERN EUROPEAN',
 'WHITE - OTHER EUROPEAN',
 'WHITE - RUSSIAN',
        'PORTUGUESE']
replacing_ethnicities = [NATIVE, ASIAN, BLACK, OTHER, HISPANIC_LATINO, UNKNOWN, WHITE]
replacing_ethnicities_str = ['NATIVE', 'ASIAN', 'BLACK', 'OTHER', 'HISPANIC_LATINO', 'UNKNOWN', 'WHITE']

#'PATIENT DECLINED TO ANSWER' are left as is

In [ ]:
#apply categorisation of ethnicities to dataframe
counter = 0
for i in replacing_ethnicities:
    train_data = replace(train_data, 'ETHNICITY', i, replacing_ethnicities_str[counter])
    test_data = replace(test_data, 'ETHNICITY', i, replacing_ethnicities_str[counter])
    counter += 1

In [ ]:
train_data['ETHNICITY'].value_counts()

### Religion


In [ ]:
train_data['RELIGION'].value_counts()

In [ ]:
#crude categorisation of religions to reduce dimensionality 
religion_other = ['HEBREW', 'UNITARIAN-UNIVERSALIST', 'HINDU', 'GREEK ORTHODOX',"JEHOVAH'S WITNESS", "BUDDHIST", 'MUSLIM', 'OTHER', 'CHRISTIAN SCIENTIST', 'EPISCOPALIAN', 'ROMANIAN EAST. ORTH', '7TH DAY ADVENTIST'] 
train_data = replace(train_data, 'RELIGION', religion_other, 'OTHER')
test_data = replace(test_data, 'RELIGION', religion_other, 'OTHER')

In [ ]:
train_data['RELIGION'].value_counts()

### Investigate relationship between outcome and categorical variables

In [ ]:
list_of_dfs = []
categorical_check = pd.DataFrame(columns = ['HOSPITAL_EXPIRE_FLAG', 'Sub-category', 'Counts', 'Sum', '%', 'Variable'])
for i in categorical_features:
    new_col_count = str(i+'_count')
    new_col_sum = str(i+'sum')
    counts = train_data.groupby('HOSPITAL_EXPIRE_FLAG').agg({i: 'value_counts'}).rename(columns = {i: new_col_count}).reset_index()
    totals = counts.groupby(i).agg({new_col_count: sum}).rename(columns = {new_col_count: new_col_sum}).reset_index()
    counts = pd.merge(counts, totals, how ='left', on=i)
    counts['%'] = counts[new_col_count]/counts[new_col_sum]
    counts['Variable'] = i
    counts.columns = categorical_check.columns
    categorical_check = pd.concat([categorical_check, counts], ignore_index=True)

In [ ]:
for i in categorical_features:
    print(categorical_check[(categorical_check['HOSPITAL_EXPIRE_FLAG'] == 1) & (categorical_check['Variable'] == i)].sort_values(by='%', ascending = False))

Summary:

- Limited/no variation by type: Gender
- Admission_type: Emergency = higher likelihood of death
- Insurance: Self-pay/medicare = higher likelihood of death
- Religion, marital_status, ethnicity: unknown categorisation = higher likelihood of death
- First care unit: CSRU = lower likelihood of death

In [ ]:
train_data[categorical_features].nunique()

Have managed to reduce dimensionality so will one_hot_encode for all features here

In [ ]:
categorical_check[(categorical_check['HOSPITAL_EXPIRE_FLAG'] == 1)].sort_values(by='%', ascending=False).head(30)

This is weird and I cannot think of an obviously good imputation method, so I drop. One cool idea (I wasn't sure how to execute) 

This provides a good justification for trying a range of encoders.

To note that we may have some data missing at ... since unknown ethnicity/religion/marital status have high proportions of death. I choose to actually turn these into an NA, and then iteratively impute them using on

# Step 2: Dealing with missing values

In [ ]:
#report nas
train_data.dtypes

In [ ]:
#report nas
test_data.isnull().sum()

## Numerical features

In [ ]:
train_data[numerical_features].isnull().sum()

Given the high correlations between groups of variables, the first impression is that imputation could be helpful. For example, imagine we had heart rate min and max, but not mean. A simple regression may help us accurately impute the mean. The same applies across all variables, and then in combination with the iterative imputer we could hopefully produce some reasonable imputations. 

In [ ]:
#group
heart_features = ['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean']
bp_features = ['SysBP_Min','SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean','MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean']
resp_features = ['RespRate_Min','RespRate_Max', 'RespRate_Mean']
temp_features = ['TempC_Min', 'TempC_Max', 'TempC_Mean']
oxygen_features = ['SpO2_Min', 'SpO2_Max', 'SpO2_Mean']
glucose_features = ['Glucose_Min', 'Glucose_Max','Glucose_Mean']

In [ ]:
train_data[heart_features+bp_features+resp_features+temp_features+oxygen_features+glucose_features].isnull().sum(axis=1).value_counts()

However, here we see that for those observations with missing values, a high proportion are actually missing data across all numerical features. The exception seems to be the glucose variables since there are far fewer missing values. 

This has implications for our imputation strategy. This provides a motivation to use the iterative imputer, using the glucose features as the starting point. 

## Marital status

In [ ]:
marital_unknown = train_data['MARITAL_STATUS'] == 'UNKNOWN (DEFAULT)'
ethnicity_unknown = train_data['ETHNICITY'] == 'UNKNOWN'
religion_unknown = train_data['RELIGION'] == 'UNOBTAINABLE'
religion_unspecified = train_data['RELIGION'] == 'NOT SPECIFIED'
marital_na = train_data['MARITAL_STATUS'].isna()

print(len(train_data[marital_na]))
print(len(train_data[marital_na&(religion_unknown|ethnicity_unknown)]))

In [ ]:
train_data[marital_na]['HOSPITAL_EXPIRE_FLAG'].sum()/len(train_data[marital_na])

Implications for imputation method. A simple method would be to impute the modal category but this doesn't seem sensible here given that a) high proportion of deaths for those with na and b) we notice those with an NA for marital status also tend to have classified relgion/ethnicity as unknown. 

As a result I classify these individuals into their own category - "Missing". 

In [ ]:
train_data['MARITAL_STATUS'].fillna('MISSING', inplace = True)
test_data['MARITAL_STATUS'].fillna('MISSING', inplace = True)
train_data['MARITAL_STATUS'].value_counts()

# Step 3: Generate new features

## 3a: Comorbidities

In [ ]:
comorbidities.isnull().sum()

In the the comorbidities data we have some nas for SEQ_NUM (refers to number of additional comorbidities diagnosed for a given HADM_ID), and ICD9_CODE. Let's investigate a bit further. 

In [ ]:
print(comorbidities[comorbidities['SEQ_NUM'].isnull() == True])
print(comorbidities[comorbidities['SEQ_NUM'].isnull() == True].shape)
comorbidities.shape

So we have 47 rows with nas out of 651,047 which is very few. In any case, let's take the first subject_id as an example. 

In [ ]:
comorbidities[comorbidities['SUBJECT_ID'] == 9998]

In other words, the nas represent individuals who have gone to hospital but have not received a diagnosis. However, this doesn't raise an issue since we will only be considering the no of comorbidities for each hospital admission. We see this later when applying the group by.

In [ ]:
#Use group by to get the no. of comorbitidies per hospital admission
comorbidities_count = comorbidities.groupby(['SUBJECT_ID','HADM_ID'], as_index = False).agg({'ICD9_CODE': 'nunique'})
#rename columns for readability 
comorbidities_count.rename(columns={'ICD9_CODE': 'No_comorbs'}, inplace=True)

Below shows that we capture the correct number of diagnoses per hospital admission.

In [ ]:
#confirm no need to remove nas
comorbidities_count[comorbidities_count['SUBJECT_ID']==9998]

In [ ]:
#merge with dataset
comorbidities_count.columns= comorbidities_count.columns.str.lower()
train_data = train_data.merge(comorbidities_count[['subject_id', 'hadm_id', 'no_comorbs']], on=['subject_id', 'hadm_id'])
test_data = test_data.merge(comorbidities_count[['subject_id', 'hadm_id', 'no_comorbs']], on=['subject_id', 'hadm_id'])

In [ ]:
train_data.columns

In [ ]:
numerical_features = numerical_features+['no_comorbs']

## 3b: Diagnosis

In [ ]:
total_survived = train_data['HOSPITAL_EXPIRE_FLAG'].value_counts()[0]
total_died = train_data['HOSPITAL_EXPIRE_FLAG'].value_counts()[1]

In [ ]:
total_diagnosed = train_data['ICD9_diagnosis'].value_counts().reset_index().rename(columns = {'ICD9_diagnosis': 'no_diagnosed', 'index': 'ICD9_diagnosis'})

In [ ]:
by_diagnosis = train_data.groupby('ICD9_diagnosis').agg({'HOSPITAL_EXPIRE_FLAG': 'sum'}).sort_values(by = 'HOSPITAL_EXPIRE_FLAG', ascending = False).rename(columns = {'HOSPITAL_EXPIRE_FLAG': 'died'}).reset_index()
by_diagnosis = pd.merge(by_diagnosis, total_diagnosed, how = 'left', on ='ICD9_diagnosis')
by_diagnosis['survived'] = by_diagnosis['no_diagnosed'] -  by_diagnosis['died']
by_diagnosis['%_of_total_dead'] = by_diagnosis['died']/total_died
by_diagnosis['%_of_diagnosed_dead'] = by_diagnosis['died']/by_diagnosis['no_diagnosed']

In [ ]:
by_diagnosis.sort_values(by = '%_of_total_dead', ascending = False).head(30)

In [ ]:
min_dead = 10
condition1 = by_diagnosis['died']>min_dead
print(by_diagnosis[by_diagnosis['died']>min_dead].sort_values(by = '%_of_diagnosed_dead', ascending = False).shape)
by_diagnosis[condition1].sort_values(by = '%_of_diagnosed_dead', ascending = False)

In [ ]:
sns.displot(by_diagnosis[by_diagnosis['died']>5].sort_values(by = '%_of_diagnosed_dead', ascending = False)['%_of_diagnosed_dead'])

In [ ]:
anyone_died = list(by_diagnosis[by_diagnosis['died']>0]['ICD9_diagnosis'].unique())
percentage_total_dead = by_diagnosis[['ICD9_diagnosis', '%_of_total_dead']]
percentage_deadly_diagnosis = by_diagnosis[condition1][['ICD9_diagnosis', '%_of_diagnosed_dead']]

In [ ]:
#include % total dead as a score
train_data = pd.merge(train_data, percentage_total_dead, on = 'ICD9_diagnosis', how = 'left')
train_data = pd.merge(train_data, percentage_deadly_diagnosis, on = 'ICD9_diagnosis', how = 'left')
#give a 0 if nobody diagnosed with disease died
train_data['%_of_diagnosed_dead'].fillna(0, inplace=True)


In [ ]:
train_data['anyone_died'] = pd.Series(dtype = object)
for i in range(0, len(train_data['ICD9_diagnosis'])):
    if train_data['ICD9_diagnosis'][i] in anyone_died:
        train_data.at[i, 'anyone_died'] = 'Someone died'
    else:
        train_data.at[i, 'anyone_died'] = 'Noone died'
        

In [ ]:
#include % total dead as a score
test_data = pd.merge(test_data, percentage_total_dead, on = 'ICD9_diagnosis', how = 'left')
test_data = pd.merge(test_data, percentage_deadly_diagnosis, on = 'ICD9_diagnosis', how = 'left')
#give a 0 if nobody diagnosed with disease died
test_data['%_of_diagnosed_dead'].fillna(0, inplace=True)
test_data['anyone_died'] = pd.Series(dtype = object)
#for loop
for i in range(0, len(test_data['ICD9_diagnosis'])):
    if test_data['ICD9_diagnosis'][i] in anyone_died:
        test_data.at[i, 'anyone_died'] = 'Someone died'
    else:
        test_data.at[i, 'anyone_died'] = 'Noone died'

In [ ]:
added_numerical_features = ['%_of_total_dead', '%_of_diagnosed_dead']
added_categorical_features = ['anyone_died']

In [ ]:
numerical_features = numerical_features + added_numerical_features
categorical_features = categorical_features + added_categorical_features

## KNN

### Pipeline and fit

numerical_features
numerical_transformer_iterative = Pipeline(
    steps=[("imputer", IterativeImputer(random_state=0, missing_values = np.nan, initial_strategy = 'mean', max_iter=30,imputation_order = 'ascending', add_indicator=True)), ("scaler", StandardScaler())]
)

#see below for simple imputer which was also tried with strategy = 'mean' and strategy = 'median'
#numerical_transformer_simple = Pipeline(
    #steps=[("imputer", SimpleImputer(missing_values = np.nan, strategy = 'mean', add_indicator=True)), ("scaler", StandardScaler())])


categorical_features
categorical_transformer_one_hot = OneHotEncoder(drop='if_binary',handle_unknown="ignore")
categorical_transformer_target = TargetEncoder() #target encoder
categorical_transformer_WOE = WOEEncoder(regularization = 0.2, randomized=True) #weight of evidence encoder
#set handle_unknown to ignore, since if it encounters an unseen categorical type in the test set, it will automatically create a column of 0s 

diagnosis_features = ['ICD9_diagnosis']
diagnosis_transformer_binary = BinaryEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer_iterative, numerical_features),
        ("cat", categorical_transformer_target, categorical_features),
        #("diag", diagnosis_transformer_binary, diagnosis_features)
    ]
)

knn_estimator = KNeighborsClassifier(algorithm='auto', weights='distance') 

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
knn_pipeline = imbPipeline([("preprocessor", preprocessor), ('sampling', SMOTE()), ("classifier", knn_estimator)]
)

grid_values = [{'classifier__n_neighbors':[50,100,200,300,400, 500],
               #'classifier__weights':['uniform', 'distance'], 
                'classifier__p':[1, 2]}] #distance consistently outperforms uniform so commented out
grid_knn = HalvingGridSearchCV(knn_pipeline, param_grid = grid_values, scoring = 'roc_auc', cv = 5, verbose = 3) #used to narrow down range for full GridSearch. First run used 


grid_knn.fit(train_data[numerical_features+categorical_features+diagnosis_features], train_data[target])

# save model
#filename = 'Numerical_iterativeimp__categorical__onehotenc.sav'
filename = 'Numerical_iterativeimp__categorical__targetenc.sav'
pickle.dump(grid_knn, open(filename, 'wb'))

print('best parameters:', grid_knn.best_params_)
print('best score:', grid_knn.best_score_)

### Results and reweight

#get probabilities 
# load the model from disk and use it
loaded_model = pickle.load(open(filename, 'rb'))
knn_train_probs = loaded_model.predict_proba(train_data)
knn_test_probs = loaded_model.predict_proba(test_data)
knn_probs = {'train_probs': knn_train_probs, 'test_probs': knn_test_probs}

knn_probs

q1 = train_data[target].sum()[0]/len(train_data[target])
r1 = 0.5
for key, value in knn_probs.items():
    if len(np.unique(knn_probs[key])) > 2:
        #necessary because sometimes we only get probabilities of 1 and 0 
        knn_probs[key] = pd.DataFrame(knn_probs[key]).apply(reweight_binary,args=(q1,r1))

#plot confusion matrix 
class_labels = ["Survived","Died"]
labels = [0,1]
cm = confusion_matrix(y_pred=knn_probs['train_probs'][:, 1], y_true=train_data[target], labels=labels)
plot_confusion_matrix(cm, class_labels)

Definitely overfitting

# Produce .csv for kaggle testing 
os.chdir(kaggle_path)
test_predictions_submit = pd.DataFrame({"icustay_id": test_data["icustay_id"], "HOSPITAL_EXPIRE_FLAG": knn_probs['test_probs'][1]})
test_predictions_submit.to_csv("test_predictions_KNN_submit.csv", index = False)

## SVM

In [ ]:
numerical_transformer_iterative = Pipeline(
    steps=[("imputer", IterativeImputer(
random_state=0, missing_values = np.nan, initial_strategy = 'mean', max_iter=30,imputation_order = 'ascending', add_indicator=True)), ("scaler", StandardScaler())]
)
#KNNImputer(missing_values=np.nan, n_neighbors = 100, weights = 'distance', add_indicator = False)
#(estimator = KNeighborsRegressor(n_neighbors=50, weights='distance', algorithm = 'auto'),
#see below for simple imputer which was also tried with strategy = 'mean' and strategy = 'median'
#numerical_transformer_simple = Pipeline(
    #steps=[("imputer", SimpleImputer(missing_values = np.nan, strategy = 'mean', add_indicator=True)), ("scaler", StandardScaler())])


categorical_transformer_one_hot = OneHotEncoder(drop='if_binary',handle_unknown="ignore")
categorical_transformer_target = TargetEncoder() #target encoder
categorical_transformer_WOE = WOEEncoder(regularization = 0.2, randomized=True) #weight of evidence encoder
#set handle_unknown to ignore, since if it encounters an unseen categorical type in the test set, it will automatically create a column of 0s 

diagnosis_features = ['ICD9_diagnosis']
diagnosis_transformer_binary = BinaryEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer_iterative, numerical_features),
        ("cat", categorical_transformer_target, categorical_features),
        #("diag", diagnosis_transformer_binary, diagnosis_features)
    ]
)


svm_estimator = SVC(probability = True, class_weight = 'balanced', random_state = 5) #class_weight set to balanced since we have an unbalanced dataset. Could have grid searched for none vs balanced, but did not due to time constraints. 

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
svm_pipeline = Pipeline([("preprocessor", preprocessor), ("classifier", svm_estimator)])

#'classifier__C':[0.01, 0.1, 1, 10]
#'classifier__gamma':[0.01, 0.1, 0.3], 
grid_values = [{'classifier__kernel': ['rbf'],'classifier__C':[0.1, 1, 2],
               'classifier__gamma':[0.005, 0.01, 0.02, 0.05, 0.075], }]
grid_svm = HalvingGridSearchCV(svm_pipeline, param_grid = grid_values, scoring = 'roc_auc', cv = 3, verbose = 3)


In [ ]:
grid_svm.fit(train_data[numerical_features+categorical_features], train_data[target])

In [ ]:
# save model
#filename = 'Numerical_iterativeimp__categorical__onehotenc.sav'
filename = 'Numerical_iterativeimp__categorical_targetenc__addedfeatures1__gridgamma__SVM.sav'
pickle.dump(grid_svm, open(filename, 'wb'))

In [ ]:
print('best parameters:', grid_svm.best_params_)
print('best score:', grid_svm.best_score_)

In [ ]:
### Results and reweight

In [ ]:
#get probabilities 
# load the model from disk and use it
loaded_model = pickle.load(open(filename, 'rb'))
svm_train_probs = loaded_model.predict_proba(train_data)
svm_test_probs = loaded_model.predict_proba(test_data)
svm_probs = {'train_probs': svm_train_probs, 'test_probs': svm_test_probs}

In [ ]:
svm_probs

In [ ]:
q1 = train_data[target].sum()[0]/len(train_data[target])
r1 = 0.5
for key, value in svm_probs.items():
    if len(np.unique(svm_probs[key])) > 2:
        #necessary because sometimes we only get probabilities of 1 and 0 
        svm_probs[key] = pd.DataFrame(svm_probs[key]).apply(reweight_binary,args=(q1,r1))

In [ ]:
svm_probs['train_probs']

In [ ]:
#plot confusion matrix 
class_labels = ["Survived","Died"]
labels = [0,1]
cm = confusion_matrix(y_pred=svm_train_probs[:, 1], y_true=train_data[target], labels=labels)
plot_confusion_matrix(cm, class_labels)

Definitely overfitting

In [ ]:
# Produce .csv for kaggle testing 
os.chdir(kaggle_path)
test_predictions_submit = pd.DataFrame({"icustay_id": test_data["icustay_id"], "HOSPITAL_EXPIRE_FLAG": svm_test_probs[:,1]})
test_predictions_submit.to_csv("test_predictions_svm_submit_added_features1_gridgamma.csv", index = False)

## Graveyard

In [ ]:
#1 Preprocessing 

##a

###i - make dummies for categorical features

for_dummy = categorical_features
for_imputation = numerical_features

#dummy = pd.get_dummies(train_data[categorical_features], drop_first = True)
dummy = make_column_transformer((OneHotEncoder(drop = 'if_binary', handle_unknown = 'ignore'), for_dummy), remainder = 'passthrough') #set handle_unknown to ignore, since if it encounters an unseen categorical type in the test set, it will automatically create a column of 0s 

###ii - iterative imputer for numerical features
imputer = make_column_transformer((IterativeImputer(random_state=0, missing_values = np.nan, initial_strategy = 'mean', max_iter=30,imputation_order = 'ascending', add_indicator=True), for_imputation), (StandardScaler(), for_imputation), remainder = 'passthrough') #ascending i.e. start with columns with fewest missing values 
#imputer = make_column_transformer((SimpleImputer(missing_values = np.nan, strategy = 'mean', add_indicator=True), for_imputation), (StandardScaler(), for_imputation), remainder = 'passthrough') #ascending i.e. start with columns with fewest missing values 

#2 estimator 
knn_estimator = KNeighborsClassifier(algorithm='auto', weights='distance') 
#choose not to grid search over different algorithms since sklearn selects the most appropriate, but am aware there are multiple options. 

#3 pipeline
knn_pipeline = imbPipeline([#('dummy', dummy), 
                    ('imputer', imputer),
                    ('scale', StandardScaler())
                            ('sampling', SMOTE()),
                            ('classifier', knn_estimator)])

grid_values = [{'classifier__n_neighbors':[1]#,10,50,100,200, 500],
               ,'classifier__weights':['uniform', 'distance'], 'classifier__p':[1]}]
grid_knn = GridSearchCV(knn_pipeline, param_grid = grid_values, scoring = 'roc_auc', cv = 5, verbose = 3)
   

In [ ]:
grid_knn.fit(train_data_check.drop(target, axis=1), train_data[target])

In [ ]:

imputer__estimator = BayesianRidge(),
    DecisionTreeRegressor(max_features='sqrt', random_state=0),
    ExtraTreesRegressor(n_estimators=20, random_state=0),
    KNeighborsRegressor(n_neighbors=2)


numeric_transformer_iterative = Pipeline(
    steps=[("imputer", (random_state=0, estimator=my_estimator, max_iter=30,add_indicator=True)), ("scaler", StandardScaler())]
)

numeric_transformer_simple = Pipeline(steps=[("imputer", SimpleImputer(strategy="mean", add_indicator = True)), ("scaler", StandardScaler())]
)

categorical_features = 
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

knn_preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),])

#make dummies
train_data = one_hot_encode(['ETHNICITY'], train_data)
test_data = one_hot_encode(['ETHNICITY'], test_data)
                                                train_data = one_hot_encode(['GENDER', 'ADMISSION_TYPE', 'INSURANCE', 'MARITAL_STATUS',
                               'FIRST_CAREUNIT', 'RELIGION'], train_data)
test_data = one_hot_encode(['GENDER', 'ADMISSION_TYPE', 'INSURANCE', 'MARITAL_STATUS',
                               'FIRST_CAREUNIT', 'RELIGION'], test_data)
print(train_data.shape)
print(test_data.shape)
                                                
                                                
#2 Imputer 

#3 Estimator 


#4 Make pipeline 

pipe_knn = imbPipeline([('dummy', column_dummy), 
                ('imputer', KNNImputer(missing_values=np.nan, n_neighbors = 100, weights = 'distance', add_indicator = False)),
                ('preprocessing', preprocessing.StandardScaler()),
                ('sampling', SMOTE()),
                #('features', fs.RFECV(estimator = DecisionTreeClassifier(class_weight = 'balanced'),
                #                      step = 10, cv = 5, scoring = 'roc_auc', verbose = 0)),
                ('classifier', KNeighborsClassifier(n_neighbors = 20,
                                                    weights = 'distance',
                                                    algorithm = 'auto'))])
knn_pipeline = imbPipeline([("preprocessor", knn_preprocessor), ("classifier", knn_estimator)])

#5 Run GridSearch 

grid_values = [#{'imputer__n_neighbors':[100]},
               {'classifier__n_neighbors':[1,5,10,50,100,200, 500],
               'classifier__weights':['uniform', 'distance'], 'classifier__p':[1, 2]}]
grid_knn = GridSearchCV(knn_pipeline, param_grid = grid_values, scoring = 'roc_auc', cv = 5, verbose = 3)
   

    # load the model from disk and use it


#6 Train, fit and save model 
grid_knn.fit(X_train, y_train)
filename = 'my_model.sav'
pickle.dump(my_model, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model.predict_proba(xtest)

In [ ]:
for i in categorical_features:
    print(train_data.groupby(i).agg({'HOSPITAL_EXPIRE_FLAG': 'value_counts'}))

So we have some very highly correlated features. With regard to missing values imputation this should prove useful. Simply imputing using the mean/median is a valid approach. However, it is reasonably crude. Take heart rate as an example - not logical to use the mean/median, especially given the high degree of correlation between variables. 

Note that I assume we are in a 'Missing at Random' world. 

In [ ]:
#for each variable, I select the variables which exhibit obvious correlations and investigate the type of correlation by visual inspection
corr[(corr.iloc[:, :]>0.2) | (corr.iloc[:, :] <-0.2)]
highly_correlated = {}
for i in corr.columns:
    correlated_vars = corr[corr[i]>0.2].index
    highly_correlated[i] = correlated_vars

In [ ]:
highly_correlated

In [ ]:
corr[corr['HeartRate_Min']>0.2].index

In [ ]:
sns.pairplot(train_data[numerical_features+target], hue = 'HOSPITAL_EXPIRE_FLAG', diag_kind = 'kde', palette = 'bright')

In [ ]:
train_data[['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean']].isnull().sum(axis=1).value_counts()

In [ ]:
train_data[['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean']]

In [ ]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
        
        """Impute values for categorical features where data was recorded as unknown with other categorical features in dataset"""
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

data = [
    ['a', 1, 2],
    ['b', 1, 1],
    ['b', 2, 2],
    [np.nan, np.nan, np.nan]
]

X = pd.DataFrame(data)
xt = DataFrameImputer().fit_transform(X)

In [ ]:
train_data[['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean']].isnull()

In [ ]:
train_data[['SysBP_Min','SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean','MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean']].isnull().sum(axis=1).value_counts()

In [ ]:
train_data[['RespRate_Min',
       'RespRate_Max', 'RespRate_Mean']].isnull().sum(axis=1).value_counts()

In [ ]:
train_data[['TempC_Min', 'TempC_Max', 'TempC_Mean']].isnull().sum(axis=1).value_counts()

In [ ]:
train_data.columns

In [ ]:
train_data[['SpO2_Min', 'SpO2_Max', 'SpO2_Mean']].isnull().sum(axis=1).value_counts()

In [ ]:
train_data[['Glucose_Min', 'Glucose_Max',
       'Glucose_Mean']].isnull().sum(axis=1).value_counts()

In [ ]:
train_data[['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min',
       'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean',
       'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min',
       'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean',
       'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max',
       'Glucose_Mean']].isnull().sum(axis=1).value_counts()

In [ ]:
diagnosis_cumulative = np.cumsum(train_data['ICD9_diagnosis'].value_counts(normalize=True).sort_values(ascending=False))
px.area(
x=range(1, diagnosis_cumulative.shape[0]+1), 
y = diagnosis_cumulative,
labels={"x": "diagnosis", "y": "Proportion of patients"})

In [ ]:
by_diagnosis

In [ ]:
comorbidities.merge(by_diagnosis[['ICD9_diagnosis', '%_of_total_dead', '%_of_diagnosed_dead']], on = ['ICD9_diagnosis'], how='left')